<a href="https://colab.research.google.com/github/kjh1311990/face/blob/main/Fuzzy_Extractor2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install komm

Mounted at /content/drive
     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for komm: filename=komm-0.7.1-cp36-none-any.whl size=56607 sha256=3baf708304b6982f3fc47b367aafd34578508061a5692c079d55be4854893398
  Stored in directory: /root/.cache/pip/wheels/22/1e/97/4a65e9a6e202aa360d36ec6a822b31cea491e2ec55788cf71f
Successfully built komm


In [2]:
import komm
import numpy as np
code = komm.BCHCode(11,365)

PARAMS_BCH_N = 2047
PARAMS_BCH_K = 133

In [3]:
import numpy as np

PARAMS_M = PARAMS_BCH_N
PARAMS_N = 512

PARAMS_LOGQ = 9
PARAMS_LOGL = 6
PARAMS_LOGT = 2

PARAMS_Q = 1 << PARAMS_LOGQ
PARAMS_L = 1 << PARAMS_LOGL

PARAMS_TRAINING_SET = 8
PARAMS_FACTOR = 1.5

In [4]:
def EMBLEM_Encode(m, shift):
    encm = ((m << 1) + 1) << (shift - 1);
    return encm.reshape(PARAMS_M, 1);

def EMBLEM_Decode(m, shift):
    decm = m >> shift;
    return decm.reshape(PARAMS_M, 1);

In [5]:
from keras.models import load_model
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import dlib, cv2

def Setup():
  A = np.random.randint(low = -PARAMS_L/2, high = PARAMS_L/2+1, size = PARAMS_N * PARAMS_M, dtype = int)
  A.resize(PARAMS_M, PARAMS_N)

  model = load_model('/content/drive/MyDrive/Notebooks/weights.h5')
  model2 = Model(model.input, model.layers[-2].output)

  return (A, model2);

def Encap(A, model, folder):
  img = np.zeros((PARAMS_TRAINING_SET,224,224,3));
  feature = np.zeros((PARAMS_TRAINING_SET,512));

  for i in range(0,PARAMS_TRAINING_SET):
    
    detector = dlib.get_frontal_face_detector();
    sp = dlib.shape_predictor('/content/drive/MyDrive/Notebooks/shape_predictor_5_face_landmarks.dat')
    tmp = dlib.load_rgb_image(folder + '/' + str(i) +'.jpg')
    tmp_detection = detector(tmp, 1)
    tmp_shape = sp(tmp, tmp_detection[0])
    tmp_aligned = dlib.get_face_chip(tmp, tmp_shape)
    tmp_aligned = cv2.cvtColor(tmp_aligned, cv2.COLOR_BGR2RGB)
    cv2.imwrite('tmp.jpg', tmp_aligned)

    img[i] = image.load_img('tmp.jpg', target_size=(224, 224))
    x = image.img_to_array(img[i]);
    x = np.expand_dims(x, axis=0);
    x = preprocess_input(x);
    feature[i] = model.predict(x);
    !rm tmp.jpg
  t = (feature.max(axis=0) - feature.min(axis=0));

  for i in range(0,PARAMS_TRAINING_SET):
    for j in range(0,512):
      if t[j] > 1:
        feature[i][j] = (feature[i][j] / (PARAMS_FACTOR * t[j]))

  #feature = feature.astype(int);

  
  b = feature.mean(axis=0).astype(int).reshape(PARAMS_N,1);
  #b = feature[0]

  m = np.random.randint(low = 0, high = 2, size = PARAMS_BCH_K, dtype = int);

  encm = code.encode(m);
  encm.reshape(PARAMS_M,1);

  c = np.remainder(np.matmul(A,b) + EMBLEM_Encode(encm, PARAMS_LOGQ-1), PARAMS_Q);


  c = (c + (1 << (PARAMS_LOGQ - PARAMS_LOGT - 1))) >> (PARAMS_LOGQ - PARAMS_LOGT);

  return (t, c, m, b)

def Decap(A, model, t, c, imgg):

  detector = dlib.get_frontal_face_detector();
  sp = dlib.shape_predictor('/content/drive/MyDrive/Notebooks/shape_predictor_5_face_landmarks.dat')
  tmp = dlib.load_rgb_image(imgg)
  tmp_detection = detector(tmp, 1)
  tmp_shape = sp(tmp, tmp_detection[0])
  tmp_aligned = dlib.get_face_chip(tmp, tmp_shape)
  tmp_aligned = cv2.cvtColor(tmp_aligned, cv2.COLOR_BGR2RGB)
  cv2.imwrite('tmp.jpg', tmp_aligned)

  img = image.load_img('tmp.jpg', target_size=(224, 224))
  !rm tmp.jpg
  
  x = image.img_to_array(img);
  x = np.expand_dims(x, axis=0);
  x = preprocess_input(x);
  feature = model.predict(x);
  feature = feature.reshape(PARAMS_N,1);

  for j in range(0,512):
    if t[j] > 1:
      feature[j] = (feature[j] / (PARAMS_FACTOR  * t[j]))

  bprime = feature.astype(int);

  c = c << (PARAMS_LOGQ - PARAMS_LOGT);
  cprime = np.remainder(c - np.matmul(A, bprime), PARAMS_Q)
  decm = EMBLEM_Decode(cprime, PARAMS_LOGQ-1).reshape(PARAMS_M)
  mprime = code.decode(decm);
  return (mprime, bprime);

In [6]:
(A, model) = Setup();

In [7]:
(t, c, m, b) = Encap(A, model, '/content/drive/MyDrive/Notebooks/face/input/1');

In [8]:
b.reshape(512)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,

In [9]:
count = 0;
for i in range(0,512):
  if b[i] != 0:
    count = count + 1;

print(count)

38


In [ ]:
for i in range(0,16):
  (mprime, bprime)= Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/1/'+ str(i) + '.jpg');
  print(np.sum(np.abs(m - mprime)))

0
0
0
0
0
0


In [ ]:
 count = 0;
for i in range(0,512):
  if bprime[i] != 0:
    count = count + 1;

print(count)

In [ ]:
count = 0;
for i in range(0,512):
  if (b[i] - bprime[i]) != 0:
    count = count + 1;

print(count)

In [ ]:
for i in range(0,6):
  (mprime, bprime) = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/2/'+ str(i) + '.jpg');
  print(np.sum(np.abs(m - mprime)))


In [ ]:
for i in range(0,10):
  (mprime, bprime) = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/3/'+ str(i) + '.jpg');
  print(np.sum(np.abs(m - mprime)))

In [10]:
cc = np.remainder(np.matmul(A, b),PARAMS_Q) >> (PARAMS_LOGQ - 2);

In [11]:
cc

array([[3],
       [1],
       [2],
       ...,
       [2],
       [2],
       [0]])

In [13]:
np.histogram(cc, bins = np.arange(5))

(array([542, 462, 512, 531]), array([0, 1, 2, 3, 4]))